In [1]:
import pandas as pd
from datasets import load_dataset
from tqdm.notebook import tqdm

In [2]:
from datasets import list_datasets
datasets_list = list_datasets()

In [3]:
eli5 = load_dataset('eli5')

Reusing dataset eli5 (C:\Users\guruvayura.ravic\.cache\huggingface\datasets\eli5\LFQA_reddit\1.0.0\b2ce8efb4328178df4b995192f01c267979a47cf5217af40a5b12c25d377f122)


In [4]:
eli5['test_eli5'][12345]

{'q_id': '8houtx',
 'title': 'Why does water heated to room temperature feel colder than the air around it?',
 'selftext': '',
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dylcnfk', 'dylcj49'],
  'text': ["Water transfers heat more efficiently than air. When something feels cold it's because heat is being transferred from your skin to whatever you're touching. Since water absorbs the heat more readily than air, it feels colder.",
   "Air isn't as good at transferring heat compared to something like water or steel (sit on a room temperature steel bench vs. a room temperature wooden bench, and the steel one will feel more cold).\n\nWhen you feel cold, what you're feeling is heat being transferred out of you.  If there is no breeze, you feel a certain way.  If there's a breeze, you will get colder faster (because the moving air is pulling the heat away from you), and if you get into water, its quite good at pulling heat from you.   Get out of the water and ha

In [4]:
wiki40b_snippets = load_dataset('wiki_snippets', name='wiki40b_en_100_0')

Reusing dataset wiki_snippets (C:\Users\guruvayura.ravic\.cache\huggingface\datasets\wiki_snippets\wiki40b_en_100_0\1.0.0\62e96e897472062a8beaf8251bbf81606854744b489c45f1b2d28349673b50e7)


In [5]:
from transformers import AutoTokenizer , AutoModel

In [6]:
qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased')
# .to('cuda:1')
_ = qar_model.eval()

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from lfqa_utils import *
# from eli5_utils import *
import os
if not os.path.isfile('wiki40b_passages_reps_32_l-8_h-768_b-512-512.dat'):
    make_qa_dense_index(qar_model, qar_tokenizer, wiki40b_snippets['train'],device='cpu',index_name='wiki40b_passages_reps_32_l-8_h-768_b-512-512.dat')
#     , device='cuda:0'

0 116.36629915237427
50 35998.22841501236
100 41118.24510741234
150 46647.44876766205
200 53197.334443330765
250 57684.247633218765
300 74510.0343182087


KeyboardInterrupt: 

In [ ]:
faiss_res = faiss.StandardGpuResources()

wiki40b_passage_reps = np.memmap('wiki40b_passages_reps_32_l-8_h-768_b-512-512.dat',dtype='float32', mode='r',shape=(wiki40b_snippets.num_rows, 128))
wiki40b_index_flat = faiss.IndexFlatIP(128)
wiki40b_gpu_index = faiss.index_cpu_to_gpu(faiss_res, 1, wiki40b_index_flat)
wiki40b_gpu_index.add(wiki40b_passage_reps)

In [ ]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_qa_dense_index(question, qar_model, qar_tokenizer, wiki40b_snippets, wiki40b_gpu_index, device='cuda:1')

df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question] + [res['passage_text'] for res in res_list],
})
df.style.set_properties(**{'text-align': 'left'})

In [ ]:
df = pd.DataFrame(columns=['question']+['ev'+str(i) for i in range(1,11)]+['reddit_answer_'+str(i) for i in range(1,4)]+['reddit_score_'+str(i) for i in range(1,4)])
for i in tqdm(range(272634), desc ="Processing"):
    question = eli5['train_eli5'][i]['title']
    if len(question)>100:
        num_answers = len(eli5['train_eli5'][i]['answers']['text'])
        if num_answers >=3:
            answers = eli5['train_eli5'][i]['answers']['text']
            scores=eli5['train_eli5'][i]['answers']['score']
            doc, res_list = query_qa_dense_index(question, qar_model, qar_tokenizer, wiki40b_snippets, wiki40b_gpu_index, device='cuda:1')
            df.loc[i] = [question]+[res['passage_text'] for res in res_list]+answers[:3]+scores[:3

df.to_pickle(r"eli5dataset.pkl")

In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch([{'host': 'localhost', 'port': '9200'}])

import eli5_utils
from eli5_utils import make_es_index_snippets,query_es_index

if not es_client.indices.exists('wiki40b_snippets_100w'):
    make_es_index_snippets(es_client, wiki40b_snippets, index_name='wiki40b_snippets_100w')
    
    
df = pd.DataFrame(columns=['question']+['ev'+str(i) for i in range(1,11)]+['reddit_answer_'+str(i) for i in range(1,4)]+['reddit_score_'+str(i) for i in range(1,4)])
for i in tqdm(range(272634), desc ="Processing"):
    question = eli5['train_eli5'][i]['title']
    if len(question)>100:
        num_answers = len(eli5['train_eli5'][i]['answers']['text'])
#         print(num_answers)
        if num_answers >=3:
            answers = eli5['train_eli5'][i]['answers']['text']
#             print(answers)
            scores=eli5['train_eli5'][i]['answers']['score']
            doc, res_list = query_es_index(question, es_client, index_name='wiki40b_snippets_100w')
            df.loc[i] = [question]+[res['passage_text'] for res in res_list]+answers[:3]+scores[:3]
    
#     else:
#         print('Short question:',question)


df.to_pickle(r"C:\Users\guruvayura.ravic\Documents\OpenInnov\Dataset\eli5dataset.pkl")